In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox, Layout
import warnings
warnings.filterwarnings("ignore")

# dati ISTAT
data = pd.read_csv("DCSP_COLTIVAZIONI_22032024191007716.csv")

# Rimozione spazi bianchi dopo le stringhe della colonna 'Tipo dato'
data['Tipo dato'] = data['Tipo dato'].str.rstrip()

# Filtro 'Tipo dato' di interesse
data_filtered = data[data["Tipo dato"].isin(['superficie totale - ettari', 'superficie totale - are', 
                                           'superficie in produzione - ettari', 'produzione totale - quintali', 
                                           'produzione totale - migliaia di quintali', 'produzione raccolta - quintali'])]

# Selezione del territorio di interesse
Territorio = 'Italia'

# Selezione degli attributi che si vogliono osservare
attributo_1 = 'superficie totale - ettari'
attributo_2 = 'produzione totale - quintali'
#attributo_1 = 'produzione totale - quintali'
#attributo_2 = 'produzione raccolta - quintali'
#attributo_1 = 'superficie totale - ettari'
#attributo_2 = 'superficie in produzione - ettari'

data_filtered2 = data_filtered
column_of_interest = ['Tipo dato', 'Tipo di coltivazione', 'TIME', 'Value', 'Territorio']
data_filtered3 = data_filtered2[column_of_interest]

### Modo 1

In [9]:
# Calcolo valore totale per Tipo dato e TIME
summarized_df = data_filtered3.groupby(['Tipo dato', 'TIME', 'Territorio'])['Value'].sum().reset_index()

# Rimozione anno 2024 perchè mancano dei dati
summarized_df = summarized_df[summarized_df['TIME'] != 2024]

# Selezione primo attributo
attributo_1_df = summarized_df[summarized_df['Tipo dato'] == attributo_1]

# Dropdown options (list of Territories)
dropdown_options = sorted(attributo_1_df['Territorio'].unique().tolist())

# Dropdown selection
dropdown = alt.binding_select(options=dropdown_options)
#selection = alt.selection_point(fields=['Territorio'], bind=dropdown, name='Select', init={'Territorio': 'Italia'})
selection = alt.selection_point(fields=['Territorio'], bind=dropdown, name='Select', value=dropdown_options[0])

# Scatter plot based on selection
scatter_plot = alt.Chart(attributo_1_df).transform_filter(
    selection  # Apply selection filter to the data
).mark_circle().encode(
    y='Value:Q',
    x='TIME:N',
    size=alt.Size(field='Value', type='quantitative', bin=alt.BinParams(maxbins=5)),
    color=alt.Color('Value:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['TIME', 'Value']
).add_params(
    selection
).properties(
    width=600,
    height=400,
    title='superficie totale - ettari'
)

# Display the scatter plot
scatter_plot

alt.Chart(...)

### Modo 2

In [19]:
# Sample DataFrame with longer category names
df = attributo_1_df

# Dropdown options (list of Territorio)
dropdown_options = sorted(df['Territorio'].unique().tolist())

# Function to update chart1 based on selected category
def update_chart1(Territorio):
    filtered_df = df[df['Territorio'] == Territorio]
    chart = alt.Chart(filtered_df).mark_circle().encode(
        y = 'Value:Q',
        x='TIME:N',
        size=alt.Size(field='Value', type='quantitative', bin=alt.BinParams(maxbins=5)),
        color=alt.Color('Value:Q', scale=alt.Scale(scheme='blues')),
        tooltip=['TIME', 'Value']
    ).properties(
        width=600,
        height=400,
        title='superficie totale - ettari'
    )
    display(chart)

dropdown_territory = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[0],  # Set initial value
    description='Select Territory:',
    style={'description_width': 'initial'}  # Set description width to full
)

# Initial display of charts and dropdowns side by side
output1 = widgets.Output()

with output1:
    display(VBox([dropdown_territory]))
    update_chart1(dropdown_territory.value)

# Arrange the outputs (charts) horizontally using HBox layout
display(output1)

# Set up event handlers for dropdowns
def on_territory_change(change):
    with output1:
        clear_output(wait=True)
        display(VBox([dropdown_territory]))
        update_chart1(change.new)

dropdown_territory.observe(on_territory_change, names='value')

Output()

### Doppio filtro

In [87]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import VBox, HBox

# Sample DataFrame with longer category names
# Utilizzo di un DataFrame fittizio per la demo
df = summarized_df

# Dropdown options
dropdown_options1 = sorted(df['Tipo dato'].unique().tolist())
dropdown_options2 = sorted(df['Territorio'].unique().tolist())

# Function to update chart based on selected Tipo dato and Territorio
def update_chart(Tipo_dato, Territorio):
    filtered_df = df[(df['Tipo dato'] == Tipo_dato) & (df['Territorio'] == Territorio)]
    
    chart = alt.Chart(filtered_df).mark_circle().encode(
        x='TIME:N',
        y='Value:Q',
        size=alt.Size('Value:Q', bin=alt.BinParams(maxbins=5), legend=None),
        color=alt.Color('Value:Q', scale=alt.Scale(scheme='blues')),
        tooltip=['TIME', 'Value']
    ).properties(
        width=600,
        height=400,
        title=Tipo_dato + ' ' + Territorio
    )
    
    # Clear previous output and display the updated chart
    with output:
        clear_output(wait=True)
        display(chart)

# Dropdown widgets
dropdown_tipo_dato = widgets.Dropdown(
    options=dropdown_options1,
    value=dropdown_options1[0],  # Set initial value
    description='Select Tipo dato:',
    style={'description_width': 'initial'}  # Set description width to full
)

dropdown_territory = widgets.Dropdown(
    options=dropdown_options2,
    value=dropdown_options2[0],  # Set initial value
    description='Select Territory:',
    style={'description_width': 'initial'}  # Set description width to full
)

# Output widget to display the chart
output = widgets.Output()

# Function to handle dropdown changes and update the chart
def on_dropdown_change(change):
    selected_tipo_dato = dropdown_tipo_dato.value
    selected_territory = dropdown_territory.value
    update_chart(selected_tipo_dato, selected_territory)

# Observe changes in dropdowns and trigger chart update
dropdown_tipo_dato.observe(on_dropdown_change, names='value')
dropdown_territory.observe(on_dropdown_change, names='value')

# Display the dropdowns and initial chart
display(VBox([dropdown_tipo_dato, dropdown_territory]))
update_chart(dropdown_tipo_dato.value, dropdown_territory.value)

# Display the output widget containing the chart
display(output)

Output()

### Doppio grafico

In [86]:
# Selezione del territorio di interesse
Territorio = 'Italia'

# Selezione degli attributi che si vogliono osservare
attributo_1 = 'superficie totale - ettari'
attributo_2 = 'produzione totale - quintali'
#attributo_1 = 'produzione totale - quintali'
#attributo_2 = 'produzione raccolta - quintali'
#attributo_1 = 'superficie totale - ettari'
#attributo_2 = 'superficie in produzione - ettari'
data_filtered4 = data_filtered3[data_filtered3['Tipo dato'] == attributo_1]
data_filtered5 = data_filtered4[data_filtered4['Territorio'] == Territorio]
data_filtered5

total_by_year = data_filtered5.groupby('TIME')['Value'].transform('sum')
data_filtered5['Percentage'] = (data_filtered5['Value'] / total_by_year) * 100
data_filtered5['Percentage'] = data_filtered5['Percentage'].round(2)
df_sorted = data_filtered5.sort_values(by = 'Percentage', ascending=False)
df_sorted_filtered = df_sorted[df_sorted['Percentage'] > 1.00]
#df_sorted_filtered = df_sorted
df_sorted_filtered = df_sorted_filtered[df_sorted_filtered['TIME'] != 2024]
df_sorted_filtered

# Sample DataFrame with longer category names
df = df_sorted_filtered

# Dropdown options (list of TIME)
dropdown_options = sorted(df['TIME'].unique().tolist())

# Function to update chart1 based on selected category
def update_chart1(TIME1):
    filtered_df = df[df['TIME'] == TIME1]
    chart = alt.Chart(filtered_df).mark_bar(color = 'blue').encode(
        x = 'Tipo di coltivazione',
        y = 'Value',
        #y = alt.Y('Percentage', scale=alt.Scale(domain=(0, df_sorted_filtered['Percentage'].max()+1))),
        tooltip=['Tipo di coltivazione', 'Value']
    ).properties(
        width=400,
        height=300,
        title = 'superficie totale - ettari'
     )
    display(chart)

# Function to update chart2 based on selected year
def update_chart2(TIME2):
    filtered_df = df[df['TIME'] == TIME2]
    chart = alt.Chart(filtered_df).mark_bar(color = 'orange').encode(
        x = 'Tipo di coltivazione',
        y = 'Value',
        #y = alt.Y('Percentage', scale=alt.Scale(domain=(0, df_sorted_filtered['Percentage'].max()+1))),
        tooltip=['Tipo di coltivazione', 'Value']
    ).properties(
        width=400,
        height=300,
        title = 'superficie totale - ettari'
     )
    display(chart)

dropdown_category = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[0],  # Set initial value
    description='Select Year:',
    style={'description_width': 'initial'}  # Set description width to full
)

# Dropdown widget for selecting year
dropdown_year = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[1],  # Set initial value
    description='Select Year:',
    style={'description_width': 'initial'}  # Set description width to full
)

# Initial display of charts and dropdowns side by side
output1 = widgets.Output()
output2 = widgets.Output()

with output1:
    display(VBox([dropdown_category]))
    update_chart1(dropdown_category.value)

with output2:
    display(VBox([dropdown_year]))
    update_chart2(dropdown_year.value)

# Arrange the outputs (charts) horizontally using HBox layout
display(HBox([output1, output2]))

# Set up event handlers for dropdowns
def on_category_change(change):
    with output1:
        clear_output(wait=True)
        display(VBox([dropdown_category]))
        update_chart1(change.new)

def on_year_change(change):
    with output2:
        clear_output(wait=True)
        display(VBox([dropdown_year]))
        update_chart2(change.new)

dropdown_category.observe(on_category_change, names='value')
dropdown_year.observe(on_year_change, names='value')